In [10]:
import sys
import re
import requests
from lxml import html
import pandas as pd

sys.path.append("../../")

import penaltyblog as pb

In [2]:
sofifa = pb.scrapers.SoFifa()

In [3]:
player_info = sofifa.get_player_info(max_pages=1)

In [4]:
player_info.head()

,name,url,nationality,team,age,overall_rating,potential,value,wage,total,positions
id,,,,,,,,,,,
248484,Nathan Collins,/player/248484/nathan-collins/220069/,Republic of Ireland,Burnley,20,75,82,€10.5M,€21K,1622,"[CB, RB]"
253124,Matheus Luiz Nunes,/player/253124/matheus-luiz-nunes/220069/,Portugal,Sporting CP,22,79,86,€35.5M,€15K,2102,[CM]
205600,Samuel Umtiti,/player/205600/samuel-umtiti/220069/,France,FC Barcelona,27,79,79,€16M,€115K,2000,[CB]
200145,Carlos Henrique Venancio Casimiro,/player/200145/carlos-henrique-venancio-casimi...,Brazil,Real Madrid CF,29,89,89,€88M,€310K,2219,[CDM]
224411,Gonçalo Manuel Ganchinho Guedes,/player/224411/goncalo-manuel-ganchinho-guedes...,Portugal,Valencia CF,24,81,86,€39.5M,€43K,2034,"[ST, LM, CF]"


In [5]:
url = "https://sofifa.com/player/241811/sergio-gomez-martin/220067/"

r = requests.get(url)

content = r.content
tree = html.fromstring(content)

In [6]:
parser = html.HTMLParser(encoding=r.encoding)
tree = html.document_fromstring(content, parser=parser)

In [63]:
tmp = dict()
tmp["name"] = tree.xpath("//div[contains(@class, 'info')]/h1")[0].text

info = tree.xpath("//div[contains(@class, 'info')]/div")[0].text_content()
tmp["weight"] = int(re.search("[0-9]*kg", info).group(0).split("kg")[0])
tmp["height"] = int(re.search("[0-9]*cm", info).group(0).split("cm")[0])
tmp["age"] = int(re.search("[0-9]*y\.o", info).group(0).split("y.o")[0])
tmp["pos"] = [x.text for x in tree.xpath("//div[contains(@class, 'info')]/div/span")] 

pd.Series(tmp)

name      Sergio Gómez Martín
weight                     68
height                    171
age                        20
pos              [LB, RM, LM]
dtype: object

In [60]:
tmp = dict()
tmp["name"] = tree.xpath("//div[contains(@class, 'info')]/h1")[0].text

info = tree.xpath("//div[contains(@class, 'info')]/div")[0].text_content()
tmp["weight"] = int(re.search("[0-9]*kg", info).group(0).split("kg")[0])
tmp["height"] = int(re.search("[0-9]*cm", info).group(0).split("cm")[0])
tmp["age"] = int(re.search("[0-9]*y\.o", info).group(0).split("y.o")[0])
tmp["pos"] = [x.text for x in tree.xpath("//div[contains(@class, 'info')]/div/span")]

stats = tree.xpath("//div[contains(@class, 'player')]//div[contains(@class, 'block-quarter')]")
for stat in stats[:2]:
    v = int(stat.xpath("div/span")[0].text)
    k = stat.xpath("div/div")[0].text
    tmp[k] = v
    
for stat in stats[2:]:
    v = stat.xpath("div")[0].text
    k = stat.xpath("div/div/text()")[0]
    tmp[k] = v    

lis = tree.xpath("//div[contains(@class, 'block-quarter')]//ul[contains(@class, 'pl')]/li")
for li in lis:
    label = li.find("label")
    if label is None:
        continue
    label = li.xpath("label/text()")[0]
    
    span = li.find("span")
    if span is not None:
        span = li.xpath("span/text()")[0]
    else:
        span = li.xpath("text()")[0].strip()
    tmp[label] = span
    
pd.Series(tmp)

name                        Sergio Gómez Martín
weight                                       68
height                                      171
age                                          20
pos                                [LB, RM, LM]
Overall Rating                               72
Potential                                    81
Value                                     €4.9M
Wage                                       €11K
Preferred Foot                             Left
Weak Foot                                     3
Skill Moves                                   4
International Reputation                      1
Work Rate                          High/ Medium
Body Type                      Normal (170-185)
Real Face                                   Yes
Release Clause                            €8.3M
ID                                       241811
Position                                     LB
Kit Number                                   17
Joined                              Jul 

In [58]:
for ul in tree.xpath("//ul[contains(@class, 'pl')]"):
    for li in ul.xpath("li"):
        if li.xpath("label"):
            k = li.xpath("label/text()")[0]
        elif li.xpath("span[@role = 'tooltip']"):
            k = li.xpath("span[@role = 'tooltip']/text()")[0]           
        else:
            continue
            
        if li.xpath("span"):
            v = li.xpath("span/text()")[0].strip()
        if li.xpath("span"):
            v = li.xpath("span/text()")[0].strip()           
        else:
            v = li.xpath("text()")[0].strip()
        print(k, v)

Preferred Foot Left
Weak Foot 3
Skill Moves 4
International Reputation 1
Work Rate High/ Medium
Body Type Normal (170-185)
Real Face Yes
Release Clause €8.3M
ID 241811
Position LB
Kit Number 17
Joined Jul 1, 2021
Contract Valid Until 2025
Best Position CAM
Best Overall Rating 75
Crossing 77
Finishing 65
Heading Accuracy 45
Short Passing 74
Volleys 64
Dribbling 74
Curve 75
FK Accuracy 76
Long Passing 69
Ball Control 73
Acceleration 76
Sprint Speed 67
Agility 85
Reactions 70
Balance 83
Shot Power 67
Jumping 61
Stamina 77
Strength 58
Long Shots 65
Aggression 68
Interceptions 72
Positioning 72
Vision 76
Penalties 78
Composure 73
Defensive Awareness 67
Standing Tackle 67
Sliding Tackle 65
GK Diving 11
GK Handling 7
GK Kicking 9
GK Positioning 11
GK Reflexes 10
Solid Player Solid Player
Early Crosser Early Crosser
Flair Flair
Playmaker (AI) Playmaker (AI)
Technical Dribbler (AI) Technical Dribbler (AI)


In [54]:
tree.xpath("black-quer")

XPathEvalError: Invalid expression